In [30]:
!pip -q install requests


In [31]:
import requests

MODEL_NAME = "phi3:mini"

# check Ollama server is running
try:
    r = requests.get("http://localhost:11434/api/tags", timeout=5)
    r.raise_for_status()
    print("✅ Ollama server is running.")
except Exception as e:
    print("❌ Ollama server is not reachable.")
    print("Fix: run `ollama serve` in Terminal, then retry.")
    raise e


✅ Ollama server is running.


In [32]:
def ollama_generate(
    prompt: str,
    model: str = MODEL_NAME,
    max_tokens: int = 220,         # strict cap to avoid hanging
    temperature: float = 0.2,
    top_p: float = 0.9,
    context_window: int = 2048,    # keep small for speed
    timeout_sec: int = 60
) -> str:
    """
    Uses Ollama local HTTP API (stable method).
    Returns generated text response.
    """

    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {
            "num_predict": max_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "num_ctx": context_window
        }
    }

    r = requests.post(url, json=payload, timeout=timeout_sec)
    r.raise_for_status()
    data = r.json()

    return data.get("response", "").strip()


In [33]:
test_prompt = """
You are VoltAI.

RULES:
- Answer ONLY using the evidence.
- If evidence is insufficient, respond exactly: Insufficient data in knowledge base.
- Use 4 bullet points max.
- Keep response under 120 words.
- End with: Sources: ...

EVIDENCE:
[SOURCE: GlobalEVOutlook2024, YEAR: 2024]
Global EV sales rose significantly in 2023, mainly driven by China and Europe.

QUESTION:
Summarize EV sales growth in 2023.
""".strip()

response = ollama_generate(test_prompt)
print(response)


- Significant increase in global electric vehicle (EV) sales observed in 2023.

- Sales surge primarily attributed to the markets of China and Europe.

- The rise reflects growing consumer acceptance and advancements in EV technology.

- Government policies promoting eco-friendly transportation also played a role.

Sources: GlobalEVOutlook2024


In [34]:
def ollama_generate_fast(prompt: str) -> str:
    return ollama_generate(
        prompt=prompt,
        max_tokens=160,
        temperature=0.15,
        top_p=0.9,
        context_window=1536,
        timeout_sec=45
    )

print(ollama_generate_fast("Say hello in one line."))


Hello!
